# Case Study 0: Fall Creek Watershed, NY

Reads, graphs and performs sensitivity analysis for observations, parameters and models from 24,000 SWMM model runs.

This allows us to run R scripts in the Python jupyter notebook.

In [1]:
%load_ext rpy2.ipython

To calculate the objective functions for the SWMM simulated streamflow, we use the hydroGOF package in R [(Documentation)](https://cran.r-project.org/web/packages/hydroGOF/index.html). For the SWMM streamflow simulations in the Fall Creek, NY watershed, we calculated mean error, mean absolute error, mean squared error, Nase-Sutcliffe efficiency, percent bias, and root mean squared error.

In [4]:
%%R
# %load CaseStudy0_OFs.R
# Case Study 0: Fall Creek, NY

# This script calculates six objective functions from 24,000 SWMM model runs and USGS streamgage 
# data using the hydroGOF package (https://cran.r-project.org/web/packages/hydroGOF/hydroGOF.pdf)

library(dplyr)
library(hydroGOF)

# load in observation, simulation, and parameter sets
obs <- read.csv("input/observation_ts.csv", header = TRUE)
  # "time_steps" row, "index" and "value" columns 
sim <- read.csv("input/simulation_ts.csv", header = TRUE)
  # "model_runs" rows, "time_steps" columns
pars <- read.csv("input/params.csv", header = FALSE)
  # "model_runs" rows, "num_pars" columns
timestamps <- read.csv("input/timestamps.csv", header = TRUE)

model_runs <- nrow(sim)
time_steps <- ncol(sim)
num_pars <- ncol(pars)

mean_error <- array(NA, model_runs)
mean_abs_error <- array(NA, model_runs)
mean_sq_error <- array(NA, model_runs)
root_mse <- array(NA, model_runs)
p_bias <- array(NA, model_runs)
nse <- array(NA, model_runs)

for (i in 1:model_runs) {

  print(i)

  mean_error[i] <- me(sim = as.numeric(sim[i, 2:time_steps]), obs = as.numeric(obs[, 2]))
  mean_abs_error[i] <- mae(sim = as.numeric(sim[i, 2:time_steps]), obs = as.numeric(obs[, 2]))
  mean_sq_error[i] <- mse(sim = as.numeric(sim[i, 2:time_steps]), obs = as.numeric(obs[, 2]))
  root_mse[i] <- rmse(sim = as.numeric(sim[i, 2:time_steps]), obs = as.numeric(obs[, 2]))
  p_bias[i] <- pbias(sim = as.numeric(sim[i, 2:time_steps]), obs = as.numeric(obs[, 2]))
  nse[i] <- NSE(sim = as.numeric(sim[i, 2:time_steps]), obs = as.numeric(obs[, 2]))

}

OF <- as.data.frame(mean_error) %>%
  setNames("me") %>%
  dplyr:: mutate(mae = mean_abs_error,
                 mse = mean_sq_error,
                 rmse = root_mse,
                 pbias = p_bias,
                 nse = nse)

write.table(OF, "input/OF_values.txt", sep = " ", row.names = FALSE, col.names = FALSE)


Error in read.table(file = file, header = header, sep = sep, quote = quote,  : 
  invalid 'row.names' specification


In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# observations for objective functions
#df_obs = pd.read_csv('input/observation_ts.csv')

# parameters for the 24,000 model runs
df_parms = pd.read_csv('input/params.csv',header=None)
df_parms.columns = ['w', 'n_imperv', 'n_perv', 's_imperv', 's_perv', 'k_sat', 'per_routed', 'cmelt', 'Tb', 'A1', 'B1']

# model results
#df_model = pd.read_csv('input/simulation_ts.csv')

# calculated objective functions
df_OFs = pd.read_csv('input/OF_values.csv')

### 1. Graph Observed and Modeled Output

In [ ]:
# Add R code here for timeseries plotting here

### 2. Approximate Bayesian Calculation
Approximate Bayesian computation (ABC) represents the combination of model parameter values that maximize the probability of representing the observed data. ABC bypasses the evaluation of the likelihood function by approximating the likelihood function by using simulations compared to the observed data. For more information on ABC, see Engeland and Gottschalk (2002), Neuman (2003), Sunnaker et al. (2013) etc. 

The steps and requirements for approximate Bayesian calculation are: 1) Observed data has a known mean and standard deviation and the user defines a summary statistic (i.e. objective function) 2) Assume you don’t know anything about the parameters, so assume a uniform prior interval [0,1]. 3) A total of n parameters are drawn from prior, the model is simulated for each of the parameter points , which results in n sequences of simulated data. 4) Calculate the summary statistic for each sequence of simulated data 5) Calculate distance between observed and simulated transition frequencies for all parameter points. Specify some tolerance  and “keep” parameter points smaller than or equal to the summary statistic as approximate samples from the posterior.

For the python script below, specify the number of model runs, tolerance of the objective functions, number of histogram bins and the figure colors. Here we used pre-defined objective functions, but the code can be modified to calculate a variety of objective functions. The plots produced are histograms of the various parameters illustrating the difference between original modeled output and the ABC constrained parameter sets.

In [ ]:
# %load approx_bayes_calc_of_defined.py 
#!/usr/bin/env python3
"""
Created on Mon Jul  8 12:57:45 2019

@author: catiefinkenbiner
"""
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

'''
Approximate Baysian Calculation requires:
    1) observation dataset (df_obs)
    2) parameter sets (df_parms)
    3) model output (df_model)
    4) objective functions (df_OFs)
    5) tolerance
    6) number of model runs
'''

def approx_bayes_calc_OF(parms,OFs,simulations):
    keep_nse = []; keep_pbias = []; keep_rmse = []
    for i in np.arange((simulations)):
        # User can redefine tolerance and OF here
        if tolerance_rmse < OFs.iloc[i,3]:
            keep_rmse.append(parms.loc[i])
            
        if np.absolute(OFs.iloc[i,4]) < tolerance_pbias:
            keep_pbias.append(parms.loc[i])
            
        if OFs.iloc[i,5] >= tolerance_nse:
            keep_nse.append(parms.loc[i])        
    return keep_nse,keep_pbias,keep_rmse

plt.rcParams.update({'font.size': 18})
def make_histograms(df_parms,bayes_approx,bins,alpha,cc1,cc2,parameters,metric):
    plt.figure(figsize=(12,12))
    for col in np.arange(1,((df_parms.iloc[0,:]).size)):
        plt.subplot(4,3,col)
        ax = df_parms.iloc[:,col].plot.hist(bins=bins,alpha=alpha,color=cc1)    
        ax = bayes_approx.iloc[:,col].plot.hist(bins=bins,alpha=alpha,color=cc2)
        ax.set_xlabel(str(parameters[col]))    
    plt.legend(['Output','ABC'],fancybox=True)
    plt.tight_layout() 
    plt.savefig(metric+'.png',dpi=300)

def runABC(df_parms,df_OFs,runs,bins,color1,color2):
    # models with objective functions within tolerance thresholds
    results_nse,results_pbias,results_rmse = np.array(approx_bayes_calc_OF(df_parms,df_OFs,runs))
    
    # saves models with objective functions within tolerance thresholds
    bayes_approx_nse = pd.DataFrame(results_nse,columns=None)
    bayes_approx_pbias = pd.DataFrame(results_pbias,columns=None)
    bayes_approx_rmse = pd.DataFrame(results_rmse,columns=None)
    parameters = list(df_parms.columns.values)
    
    # print ABC results and make figures
    print('precent of models with NSE >= to',str(tolerance_nse),'are:',str(len(results_nse)/runs),'%')
    make_histograms(df_parms,bayes_approx_nse,bins,0.5,color1,color2,parameters,'NSE')
    print('precent of models with',str(-tolerance_pbias),'<= p-bias >=',str(tolerance_pbias),'are:',str(len(results_pbias)/runs),'%')
    make_histograms(df_parms,bayes_approx_pbias,bins,0.5,color1,color3,parameters,'pbias')
    print('precent of models with RMSE < to',str(tolerance_rmse),'are:',str(len(results_rmse)/runs),'%')
    make_histograms(df_parms,bayes_approx_rmse,bins,0.5,color1,color4,parameters,'RMSE')


In [ ]:
# Specify tolerance for objective functions (OF)
tolerance_rmse = 6.0   # OF < tolerance
tolerance_pbias = 15.0 # -tolerance < OF > tolerance
tolerance_nse = 0.0  # OF >= tolerance

runs = 24000 # specify number of model runs
bins = 100   # specify number of histogram bins
color1 = 'b' # color of original model output
color2 = 'k' # color of 1st ABC applied to OF (NSE)
color3 = 'r' # color of 2nd ABC applied to OF (p-bias)
color4 = 'g' # color of 3rd ABC applied to OF (RMSE)

# Runs function that evaluates models outputs with approximate Bayesian calculations
runABC(df_parms,df_OFs,runs,bins,color1,color2)

### 2. Sensitivity Analysis
Python modules to conduct three sensitivity analyses are included in this jupyter notebook.

Sobol sensitivity

Delta sensitivity

OLS sensitivity

write some stuff about this ...


Approximate Bayesian computation (ABC) represents the combination of model parameter values that maximize the probability of representing the observed data. ABC bypasses the evaluation of the likelihood function by approximating the likelihood function by using simulations compared to the observed data. For more information on ABC, see Engeland and Gottschalk (2002), Neuman (2003), Sunnaker et al. (2013) etc. 

The steps and requirements for approximate Bayesian calculation are: 1) Observed data has a known mean and standard deviation and the user defines a summary statistic (i.e. objective function) 2) Assume you don’t know anything about the parameters, so assume a uniform prior interval [0,1]. 3) A total of n parameters are drawn from prior, the model is simulated for each of the parameter points , which results in n sequences of simulated data. 4) Calculate the summary statistic for each sequence of simulated data 5) Calculate distance between observed and simulated transition frequencies for all parameter points. Specify some tolerance  and “keep” parameter points smaller than or equal to the summary statistic as approximate samples from the posterior.

For the python script below, specify the number of model runs, tolerance of the objective functions, number of histogram bins and the figure colors. Here we used pre-defined objective functions, but the code can be modified to calculate a variety of objective functions. The plots produced are histograms of the various parameters illustrating the difference between original modeled output and the ABC constrained parameter sets.



In [ ]:
# %load CaseStudy0_SI_RCPs.py
#!/usr/bin/env python3
"""
Created on Wed Jul 10 12:56:56 2019

@author: kylasemmendinger
"""
# import python libraries
import pandas as pd
import os

# back out a directory to load python functions from "Scripts" folder
org_dir_name = os.path.dirname(os.path.realpath('CaseStudy0_SI_RCPs.py'))
parent_dir_name = os.path.dirname(os.path.dirname(os.path.realpath('CaseStudy0_SI_RCPs.py')))
os.chdir(parent_dir_name + "/Scripts")

# load python functions from ‘Scripts’ folder
import delta
import sobol
import ols
import radial_conv_plots

# move back into case study 0 folder
os.chdir(org_dir_name)

# Define the model inputs
problem = {
    'num_vars': 11,
    'names': ['w', 'n_imperv', 'n_perv', 's_imperv', 's_perv', 'k_sat', 'per_routed', 'cmelt', 'Tb', 'A1', 'B1'],
    'bounds': [[500, 1500], # meters
               [0.01, 0.2],
               [0.01, 0.2],
               [0, 10],
               [0, 10],
               [0.01, 10],
               [0, 100],
               [0, 4],
               [-3, 3],
               [0.0001, 0.01],
               [1, 3]]
}

# load in model simulations, observation data, parameter sets (Saltelli sampled), timestamps, and objective function values
sim = pd.read_csv("input/simulation_ts.csv", index_col = 0)
obs = pd.read_csv("input/observation_ts.csv")
pars = pd.read_csv("input/params.csv", header = None)
timestamps = pd.read_csv("input/timestamps.csv")
OF = pd.read_csv("input/OF_values.csv")

# save the parameter names
param_names = problem['names']

# calculate Sobol first-, second-, and total order indices --> MUST BE BASED ON SALTELLI SAMPLING SCHEME
results_SI = []
results_SI = sobol.objective_function_sobol(problem, OF)

# create radial convergence plots based on results_SI
radial_conv_plots.radial_conv_plots(problem, results_SI, OF)

# calculate delta indices and sobol first-order indices
results_delta = []
results_delta = delta.objective_function_delta(problem, pars, OF)

# calculate R^2 from OLS regression
results_R2 = []
results_R2 = ols.objective_function_OLS(OF, pars, param_names)

### 3. Visualizations

Talk about some visualization techniques here

In [ ]:
%%R
# %load CaseStudy0_Visuals.R

# This script loads in data from Sobol, Delta, and OLS sensitivity analyses calculated in
# Python script for Case Study 0: Fall Creek, NY.

library(dplyr)

# load in observation, simulation, and parameter sets
obs <- read.csv("input/observation_ts.csv", header = TRUE)
  # "time_steps" row, 1 column of values

sim <- read.csv("input/simulation_ts.csv", header = TRUE) %>%
  dplyr::select(-1)
  # "model_runs" rows, "time_steps" columns

pars <- read.csv("input/params.csv", header = FALSE)
  # "model_runs" rows, "num_pars" columns

timestamps <- read.csv("input/timestamps.csv", header = TRUE)
  # "time_steps" row, 1 column of values

OF <- read.csv("input/OF_values.csv", header = TRUE)
  # "model_runs" rows, "num_OF" columns

# save names of objective functions and parameters
OF_names <- colnames(OF)
param_names <- c("w", "n_imperv", "n_perv", "s_imperv", "s_perv", "k_sat", 
                 "per_routed", "cmelt", "Tb", "A1", "B1")

# clean up data
obs <- array(obs[, 2])
timestamps <- array(timestamps[, 2])
colnames(pars) <- param_names

# set variables of number of model runs, time steps, and number of parameters
model_runs <- nrow(sim)
time_steps <- ncol(sim)
num_pars <- ncol(pars)
num_OF <- ncol(OF)

# load in results from delta, sobol, and ols sensitivity analyses (calculated in python script)
source("../Scripts/python_to_r_results.R")
results_sobol <- python_to_r_results(data_type = "sobol", param_names, OF_names)
results_delta <- python_to_r_results(data_type = "delta", param_names, OF_names)
results_ols <- python_to_r_results(data_type = "ols", param_names, OF_names)

# scatter plots of objective functions versus parameter values
source("../Scripts/scatterplots.R")
for (i in 1:num_OF) {
  
  # subset by objective function, i
  objective_fun <- OF[, i]

  # create scatterplots of all parameters versus objective function, i
  par_OF_scatter(params = pars, objective_fun, OF_name = colnames(OF)[i])
  
}

# portrait plots of objective functions versus parameter values
source("../Scripts/portrait_plots.R")
portrait_plot(results_sobol)
portrait_plot(results_delta)
portrait_plot(results_ols)

# spiders plots of objective functions versus parameter values
source("../Scripts/spider_plots.R")
spiderplot(results_sobol)
spiderplot(results_delta)
spiderplot(results_ols)

### XX. Conclusion

We did awesome stuff and this is how we feel about it...

### XX. References
- Engeland, K., Gottschalk L. Bayesian estimation of parameters in regional hydrological model, Hydrol. Earth Sys. Sci., **2002**, *6*(5), 883-898. https://doi.org/10.5194/hess-6-883-2002
- Neuman, S. Maximum likelihood Bayesian averaging of uncertain model predictions, Stoch. Environ. Res. Risk Assess. **2003** *17*, 291. https://doi.org/10.1007/s00477-003-0151-7
- Sunnåker M., Busetto A.G., Numminen E., Corander J., Foll M., Dessimoz C. Approximate Bayesian Computation, PLoS Comput. Biol. **2013** *9*(1): e1002803. https://doi.org/10.1371/journal.pcbi.1002803
